In [ ]:
pip install fastquant

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from fastquant import backtest,get_crypto_data,get_stock_data
import backtrader as bt
import backtrader.analyzers as btanalyzers
import numpy as np

In [ ]:
df = get_crypto_data('BTCUSDT',start_date='2021-01-01',end_date='2021-11-27', time_resolution='1h')

In [ ]:
df2 = df.copy()

In [ ]:
df2.reset_index(level=0, inplace=True)

In [ ]:
df2.head()

,dt,open,high,low,close,volume
0,2020-12-31 19:00:00,28872.24,29000.00,28742.41,28897.83,2293.821339
1,2020-12-31 20:00:00,28897.84,29139.65,28862.00,29126.70,1936.480299
2,2020-12-31 21:00:00,29126.71,29169.55,28900.79,28966.36,2524.473111
3,2020-12-31 22:00:00,28966.36,29143.73,28910.19,29100.84,1438.506326
4,2020-12-31 23:00:00,29100.83,29110.35,28780.00,28923.63,1976.419299


In [ ]:
from fastquant import CustomStrategy, BaseStrategy
from fastquant.indicators import MACD, CrossOver 
from fastquant.indicators.custom import CustomIndicator


# Create a subclass of the BaseStrategy, We call this MAMAStrategy (MACD + ALMA)
class MAMAStrategy(BaseStrategy):
    
    
    count = 0
    
    params = (
        ("upper_band", "Upper_Band"),   # name for the ALMA column from the dataframe
        ('sar_period', 2),
        ("af", 0.05),
        ("afmax",0.50),
        ("rsi_period", 14),  
        ("rsi_upper", 70),
        ("rsi_lower",50)
        )
        
    def __init__(self):
        # Initialize global variables
        super().__init__()
        
        
        self.rsi_period = self.params.rsi_period
        self.rsi_upper = self.params.rsi_upper
        self.rsi_lower = self.params.rsi_lower
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.rsi_period, upperband=self.rsi_upper, lowerband=self.rsi_lower)
        self.sar_period = self.params.sar_period
        self.af = self.params.af
        self.afmax = self.params.afmax
        
        self.ParabolicSAR = bt.indicators.ParabolicSAR(period = self.sar_period, af=self.af, afmax= self.afmax)
    

        
        print("===Strategy level arguments===")
        print("PARAMS: ", self.params)
        

    # Buy when the custom indicator is below the lower limit, and sell when it's above the upper limit
    

    def buy_signal(self):
        rsi_buy =  self.rsi < self.rsi_lower   # Close is above ALMA
        sar_buy = self.data.close[0]> self.ParabolicSAR    # MACD crosses signal line upward
        return rsi_buy and sar_buy 
   
    def sell_signal(self):
        sar_sell = self.ParabolicSAR > self.data.close[0]
        rsi_sell = self.rsi>self.rsi_upper
        return sar_sell and rsi_sell
        


In [ ]:
results,history = backtest(MAMAStrategy, df, init_cash=100000,plot=False, return_history=True, execution_type='close')

Starting Portfolio Value: 100000.00
2021-11-27T00:00:00, ===Global level arguments===
2021-11-27T00:00:00, init_cash : 100000
2021-11-27T00:00:00, buy_prop : 1
2021-11-27T00:00:00, sell_prop : 1
2021-11-27T00:00:00, commission : 0
2021-11-27T00:00:00, stop_loss : 0
2021-11-27T00:00:00, stop_trail : 0
2021-11-27T00:00:00, take_profit : 0
===Strategy level arguments===
PARAMS:  <backtrader.metabase.AutoInfoClass_LineRoot_LineMultiple_LineSeries_LineIterator_DataAccessor_StrategyBase_Strategy_BaseStrategy_MAMAStrategy1 object at 0x0000022BBCA38FA0>
2021-11-27T00:00:00, Final Portfolio Value: 192627.22
2021-11-27T00:00:00, Final PnL: 92627.22
Time used (seconds): 4.589513063430786
Number of strat runs: 1
Number of strats per run: 1
Strat names: ['MAMAStrategy']
**************************************************
--------------------------------------------------
Strategy Parameters	init_cash:100000	buy_prop:1	sell_prop:1	fractional:False	commission:0	stop_loss:0	stop_trail:0	take_profit:0	e

In [ ]:
results

,strat_id,init_cash,buy_prop,sell_prop,fractional,commission,stop_loss,stop_trail,take_profit,execution_type,...,won,lost,won_avg,won_avg_prcnt,lost_avg,lost_avg_prcnt,won_max,won_max_prcnt,lost_max,lost_max_prcnt
0,0,100000,1,1,False,0,0,0,0,close,...,13,3,16208.997692,16.208998,-26449.15,-26.44915,27710.34,27.71034,-48946.14,-48.94614


In [ ]:
results.columns

Index(['strat_id', 'init_cash', 'buy_prop', 'sell_prop', 'fractional',
       'commission', 'stop_loss', 'stop_trail', 'take_profit',
       'execution_type', 'channel', 'symbol', 'allow_short', 'short_max',
       'add_cash_amount', 'add_cash_freq', 'invest_div', 'upper_band',
       'sar_period', 'af', 'afmax', 'rsi_period', 'rsi_upper', 'rsi_lower',
       'rtot', 'ravg', 'rnorm', 'rnorm100', 'len', 'drawdown', 'moneydown',
       'max', 'maxdrawdown', 'maxdrawdownperiod', 'sharperatio', 'pnl',
       'final_value', 'total', 'win_rate', 'won', 'lost', 'won_avg',
       'won_avg_prcnt', 'lost_avg', 'lost_avg_prcnt', 'won_max',
       'won_max_prcnt', 'lost_max', 'lost_max_prcnt'],
      dtype='object')

In [ ]:
final = results[['sar_period', 'total','af','rsi_upper','rsi_lower','maxdrawdown', 'maxdrawdownperiod', 'sharperatio', 'pnl','win_rate' ]]

In [ ]:
final

,sar_period,total,af,rsi_upper,rsi_lower,maxdrawdown,maxdrawdownperiod,sharperatio,pnl,win_rate
0,2,17,0.05,70,50,48.942582,182,0.978408,92627.22,0.764706


In [ ]:
orders = history['orders']

In [ ]:
orders

,strat_id,strat_name,dt,type,price,size,value,commission,pnl
0,0,execution_typeclose,2021-01-04 06:00:00,buy,32813.01,3,98439.03,0.0,0.00
1,0,execution_typeclose,2021-01-08 00:00:00,sell,39432.28,-3,98439.03,0.0,19857.81
2,0,execution_typeclose,2021-01-10 04:00:00,buy,40316.64,2,80633.28,0.0,0.00
3,0,execution_typeclose,2021-01-10 23:00:00,buy,38428.23,1,38428.23,0.0,0.00
4,0,execution_typeclose,2021-02-09 00:00:00,sell,46374.87,-3,119061.51,0.0,20063.10
5,0,execution_typeclose,2021-02-11 00:00:00,buy,44807.58,3,134422.74,0.0,0.00
6,0,execution_typeclose,2021-02-19 00:00:00,sell,51552.60,-3,134422.74,0.0,20235.06
7,0,execution_typeclose,2021-02-19 05:00:00,buy,51166.30,3,153498.90,0.0,0.00
8,0,execution_typeclose,2021-02-21 00:00:00,sell,55841.19,-3,153498.90,0.0,14024.67
9,0,execution_typeclose,2021-02-22 08:00:00,buy,56422.92,3,169268.76,0.0,0.00


In [ ]:
import pandas as pd

In [ ]:
pd.merge(df2, orders, on='dt')

,dt,open,high,low,close,volume,strat_id,strat_name,type,price,size,value,commission,pnl
0,2021-01-11 03:00:00,36363.93,36697.97,33600.00,35438.23,18753.360082,0,execution_typeclose,buy,36363.94,2,72727.88,0.0,0.00
1,2021-02-09 00:00:00,46374.86,47498.00,46233.95,46768.22,10775.231095,0,execution_typeclose,sell,46374.87,-2,72727.88,0.0,20021.86
2,2021-04-18 08:00:00,56184.51,56498.00,54459.56,54953.65,7610.111419,0,execution_typeclose,buy,56180.34,2,112360.68,0.0,0.00
3,2021-05-01 00:00:00,57697.25,58088.00,57421.05,57811.42,3480.449516,0,execution_typeclose,sell,57694.27,-2,112360.68,0.0,3027.86
4,2021-05-13 03:00:00,50257.78,50490.09,49544.00,50461.99,6253.543763,0,execution_typeclose,buy,50255.84,2,100511.68,0.0,0.00
5,2021-06-15 00:00:00,40516.28,40900.00,40169.51,40355.88,4690.099098,0,execution_typeclose,sell,40516.29,-2,100511.68,0.0,-19479.10
6,2021-06-25 22:00:00,31799.14,31975.98,31275.00,31423.32,4173.793707,0,execution_typeclose,buy,31799.37,3,95398.11,0.0,0.00
7,2021-07-22 00:00:00,32144.51,32376.78,31878.79,32053.18,3445.789055,0,execution_typeclose,sell,32144.51,-3,95398.11,0.0,1035.42
8,2021-09-08 00:00:00,46868.57,47110.00,46412.98,47035.99,2796.571530,0,execution_typeclose,buy,46863.73,2,93727.46,0.0,0.00
9,2021-10-02 00:00:00,48141.60,48224.01,47610.76,47817.73,2213.004520,0,execution_typeclose,sell,48141.61,-2,93727.46,0.0,2555.76


In [ ]:
final.sort_values(by='pnl', ascending=False)

,sar_period,total,af,rsi_upper,rsi_lower,maxdrawdown,maxdrawdownperiod,sharperatio,pnl,win_rate
0,2,6,0.05,70,30,28.232051,198,-7.200418,243.89,0.666667
